# Project: Self-storage Companies in the UK

### Project Goals

Collect publicly available data from Companies House - GOV.UK on Self-storage companies in the UK.

### Details about  my contribution to the project

Entering the search query "self storage" at https://find-and-update.company-information.service.gov.uk/search returned a list of companies spanning 20pages. A close look at the search results on page 2 https://find-and-update.company-information.service.gov.uk/search?q=self+storage&page=2 and page 3 https://find-and-update.company-information.service.gov.uk/search?q=self+storage&page=3 revealed that:
* Each page contained the name of 20 Self-storage companies making 400 companies in all.
* The links to all 20pages of search results are identical with the exception of the page number at the end.
* Each company name is hyperlinked to respective company pages holding the data we needed.


To get the data of interest (Company name, Name of the People, Address) for each of these 400 companies, I wrote a Python script to do the followings:
1. Crawl through all 20pages of search result scraping the links to each company’s page (i.e. 20links per page x 20pages)
2. Follow everyone of these 400links, crawling through each page and scraping company name, name of people and address for each Self-storage company.
3. Saving the final result as an Excel file.

### A summary of the project's success

Completed: 21st August, 2021

### Keywords:
Web Crawling, Web Scraping, Data cleaning, Python, Programming, Pandas, NumPy, Beautiful soup, Selenium, Jupyter notebook, Chrome

### Requirements:
    BeautifulSoup
    import numpy
    time / sleep
    from random / randint
    selenium / webdriver
    selenium.common.exceptions / NoSuchElementException
    pandas
    csv
    openpyxl

### Step 1:
We start by sraping the link to each company's page from the search result. There are 20 companies per page, 20 pages in all.

In [2]:
# import libraries
from bs4 import BeautifulSoup
import numpy as np
from time import sleep
from random import randint
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
# import re
import csv
# import itertools
!pip install openpyxl

In [ ]:
# Get the url for each company, adjust the `np.arange()` to visit all 20 pages avoing "off-by-one error"
# NB: Search result is 20 pages long and each page lists 20 companies
pages = np.arange(1, 20+1, 1)
url_collected=[]

for p in pages:
    page = "https://find-and-update.company-information.service.gov.uk/search?q=self+storage&page=" +str(p)
    driver = webdriver.Chrome(r"C:\Users\user\Documents\Docs\Tech\Selenium drivers\chromedriver")
    driver.get(page)  
    sleep(randint(5,15))
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    urls = [item.get("href") for item in soup.find_all("a")]
    url_collected.append(urls)
    driver.close()

In [ ]:
# NB: url_collected is now a list of lists which needs to be made into a flat list    
url_list = [item for sublist in url_collected for item in sublist]

In [ ]:
url_list

Some data cleaning

In [ ]:
#Remove duplicates and none values
urls_final = list(dict.fromkeys(url_list))
urls_final = list(filter(None, urls_final)) 

In [ ]:
#Remove if not starting with company,
url_final = [x for x in urls_final if x.startswith('/company/')]

In [ ]:
len(url_final)

In [ ]:
url_final[:3]

In [ ]:
# Gives all the links to each company's page in full after scraing, cleaning, and reconstructing
string = 'https://find-and-update.company-information.service.gov.uk/'
final_list=[string + s for s in url_final]

This `final_list` is iterated over in order to visit each of those 400 pages to collect tha data of interested (i.e. Company name, Name of the People, Address)

In [ ]:
# Loads the links into pandas
companies_links = pd.DataFrame (final_list ,columns=['Companies links'])

In [ ]:
companies_links[:3]

In [ ]:
# Exports `companies_links` DataFrame to Excel file named Companies Links
companies_links.to_excel('./Companies Links.xlsx', index = False)

Now we reload the excel file containing all the links, and loop over the list, visiting each page and extracting `company_name`, `person`, `reg_office_add`, and `ref_link` that is, the link to the page in question for reference purposes

For some reason, the iteration only scraped the first `122` pages. Hence, I had to run the iteration in 2 passes and:
* Saved the result of the first pass (from 1 to 122 pages) as an excel file.
* Read this excel file back into pandad to get the index of the last link visited in the previous pass.
* Ran the iteration again, picking up from where it stopped the last time (from 123 to 400) to complete scrapping all target data.


In [3]:

companies_links_reloaded = pd.read_excel('./Companies Links.xlsx', sheet_name='Sheet1', header=0)

companies_links_reloaded

,Companies links
0,https://find-and-update.company-information.se...
1,https://find-and-update.company-information.se...
2,https://find-and-update.company-information.se...
3,https://find-and-update.company-information.se...
4,https://find-and-update.company-information.se...
...,...
395,https://find-and-update.company-information.se...
396,https://find-and-update.company-information.se...
397,https://find-and-update.company-information.se...
398,https://find-and-update.company-information.se...


In [4]:
links = [_ for _ in companies_links_reloaded['Companies links']]   

In [5]:
links.index('https://find-and-update.company-information.service.gov.uk//company/07194680')

122

In [6]:
# final_list2 = []
# Find the 124th element i.e https://find-and-update.company-information.service.gov.uk//company/07194680
# for every ot`her element from the 124th to 400th, append element to final_list2

final_list2 = []
for num, link in enumerate (links): 
    if num < 122:
        pass
    else:
        final_list2.append(link)


len(final_list2)

278

### Step 2: 
Now that we have the remaining links, we visit each and every one of them scrapping all the only what we need

In [8]:
company_data = {'company_name': [], 'person': [], 'reg_office_add': [], 'ref_link': []}


for coylink in final_list2:
    page = coylink
    driver = webdriver.Chrome(r"C:\Users\user\Documents\Docs\Tech\Selenium drivers\chromedriver")
    driver.get(page)  
    sleep(randint(2,5))
    
    try:   
        coy_name = driver.find_element_by_class_name('heading-xlarge')
        company_data['company_name'].append(coy_name.text)
    
    except NoSuchElementException:
        company_data['company_name'].append('NA')

    try:       
        coy_add = driver.find_element_by_xpath('//*[@id="content-container"]/dl/dd')
        company_data['reg_office_add'].append(coy_add.text)
    
    except NoSuchElementException:
        company_data['reg_office_add'].append('NA')

        
    sleep(randint(1,4))

    
    company_data['ref_link'].append(coylink)

    
    try:       
        driver.find_element_by_id('people-tab').click()
        
    except NoSuchElementException:
        company_data['person'].append('NA')

        continue
    
    
    # use the try except loop to get only the first person if there are more than one
    try:       
        people = driver.find_element_by_xpath('//*[@id="officer-name-1"]/a')
        company_data['person'].append(people.text)

    except NoSuchElementException:
        company_data['person'].append('NA')

    


    driver.close()

In [33]:
company_data

dict

### Step 3:
Piece together the collected data and export the combined DataFrame as an Excel file

In [37]:
index = [b for b in range(123, 400+1, 1)]

In [47]:
gander = company_data

In [49]:
# Load `company_data` into pandas DataFrame
remaining_table = pd.DataFrame(data = gander, columns = ['company_name', 'person', 'reg_office_add' , 'ref_link'], index = index)

In [50]:
remaining_table.head()[:5]

,company_name,person,reg_office_add,ref_link
123,ARMOURSTORE SELF STORAGE LTD,"LAYN, Steven Charles Keith Harold","Foxwood Lodge Marefen Drove, Littleport, Ely, ...",https://find-and-update.company-information.se...
124,ARMSTRONG SELF STORAGE LTD,"ARMSTRONG, Richard","40 Railway Street, Lisburn, United Kingdom, BT...",https://find-and-update.company-information.se...
125,ASHBOURNE SELF STORAGE LTD,"MORLEY, Jillian","59, Dale End Rd, 59 Dale End Road, Hilton, Der...",https://find-and-update.company-information.se...
126,ASHBY SELF STORAGE LIMITED,"EATON, Charles Matthew","Beech House Farm Ashby Road, Packington, Ashby...",https://find-and-update.company-information.se...
127,ASHFORD SELF STORAGE LIMITED,"FARMER, Peter Lester","8 High Street, Heathfield, East Sussex, TN21 8LS",https://find-and-update.company-information.se...


In [51]:
# Save DataFrame to Excel file
writer = pd.ExcelWriter('company_scrape complement.xlsx')
# write dataframe to excel
remaining_table.to_excel(writer, 'self storage companies')
# save the excel
writer.save()

Save the output in an Excel file

In [58]:
first_half = pd.read_excel('./company_scrape first 123 without double company name.xlsx', sheet_name='self storage companies', header=0)

first_half

,Unnamed: 0,company_name,person,reg_office_add,ref_link
0,0,SELF STORAGE BARNSLEY LTD,"SHARMAN, Jean","Unit 20 Fall Bank Industrial Estate, Dodworth,...",https://find-and-update.company-information.se...
1,1,SELF STORAGE CENTRE LIMITED,"ELBRO, Matthew James","Luker Bros (r&s) Ltd, 17a Shelley Close, Headi...",https://find-and-update.company-information.se...
2,2,SELF STORAGE CHESHIRE LIMITED,"BEAUMONT, Michael Dixon","St George's House, 215-219 Chester Road, Manch...",https://find-and-update.company-information.se...
3,3,SELF STORAGE COMPANY (DORSET) LIMITED,"SAMS, Louise Helen","Stafford House, 10 Prince Of Wales Road, Dorch...",https://find-and-update.company-information.se...
4,4,SELF STORAGE CONCIERGE LIMITED,"COATH, Michael","James Taylor House, St Albans Road East, Hatfi...",https://find-and-update.company-information.se...
...,...,...,...,...,...
118,118,ARDEN SELF STORAGE LIMITED,"COOKES, Rosemary Margaret","Countrywide House, 23 West Bar, Banbury, Oxfor...",https://find-and-update.company-information.se...
119,119,ARMADILLO SELF STORAGE LIMITED,"BEAVIS, Shauna","2 The Deans, Bridge Road, Bagshot, Surrey, GU1...",https://find-and-update.company-information.se...
120,120,ARMADILLO SELF STORAGE 2 LIMITED,"BEAVIS, Shauna","2 The Deans, Bridge Road, Bagshot, Surrey, GU1...",https://find-and-update.company-information.se...
121,121,ARMAGH SELF STORAGE LTD,"JAMESON, David","Barry Thompson & Company, 76-78 Church Street,...",https://find-and-update.company-information.se...


In [66]:
second_half = pd.read_excel('./company_scrape remaining 124 to 400 (complement).xlsx', sheet_name='self storage companies', header=0)

second_half

,Unnamed: 0,company_name,person,reg_office_add,ref_link
0,123,ARMOURSTORE SELF STORAGE LTD,"LAYN, Steven Charles Keith Harold","Foxwood Lodge Marefen Drove, Littleport, Ely, ...",https://find-and-update.company-information.se...
1,124,ARMSTRONG SELF STORAGE LTD,"ARMSTRONG, Richard","40 Railway Street, Lisburn, United Kingdom, BT...",https://find-and-update.company-information.se...
2,125,ASHBOURNE SELF STORAGE LTD,"MORLEY, Jillian","59, Dale End Rd, 59 Dale End Road, Hilton, Der...",https://find-and-update.company-information.se...
3,126,ASHBY SELF STORAGE LIMITED,"EATON, Charles Matthew","Beech House Farm Ashby Road, Packington, Ashby...",https://find-and-update.company-information.se...
4,127,ASHFORD SELF STORAGE LIMITED,"FARMER, Peter Lester","8 High Street, Heathfield, East Sussex, TN21 8LS",https://find-and-update.company-information.se...
...,...,...,...,...,...
273,396,JOHNSONS OF OTLEY SELF STORAGE LIMITED,"JOHNSON, Alison Ruth","Carr Bank House, Carr Bank, Otley, United King...",https://find-and-update.company-information.se...
274,397,JORVIK REMOVALS & SELF STORAGE LTD,"HUGHES, Melissa Jade","2b Kettlestring Lane, York, England, YO30 4XF",https://find-and-update.company-information.se...
275,398,J SIMPSON SELF STORAGE LIMITED,"SIMPSON, John Christopher","Hill Top East Caldwell, Richmond, North Yorksh...",https://find-and-update.company-information.se...
276,399,J28 SELF STORAGE LIMITED,"KOHLMAN, David Peter","Ruwmell Hall, Rumwell, Taunton, Somerset, TA4 1EL",https://find-and-update.company-information.se...


In [60]:
# Concatenate both DataFrames
frames = [first_half, second_half]

all_self_storage_companies_UK = pd.concat(frames)

In [61]:
all_self_storage_companies_UK

,Unnamed: 0,company_name,person,reg_office_add,ref_link
0,0,SELF STORAGE BARNSLEY LTD,"SHARMAN, Jean","Unit 20 Fall Bank Industrial Estate, Dodworth,...",https://find-and-update.company-information.se...
1,1,SELF STORAGE CENTRE LIMITED,"ELBRO, Matthew James","Luker Bros (r&s) Ltd, 17a Shelley Close, Headi...",https://find-and-update.company-information.se...
2,2,SELF STORAGE CHESHIRE LIMITED,"BEAUMONT, Michael Dixon","St George's House, 215-219 Chester Road, Manch...",https://find-and-update.company-information.se...
3,3,SELF STORAGE COMPANY (DORSET) LIMITED,"SAMS, Louise Helen","Stafford House, 10 Prince Of Wales Road, Dorch...",https://find-and-update.company-information.se...
4,4,SELF STORAGE CONCIERGE LIMITED,"COATH, Michael","James Taylor House, St Albans Road East, Hatfi...",https://find-and-update.company-information.se...
...,...,...,...,...,...
273,396,JOHNSONS OF OTLEY SELF STORAGE LIMITED,"JOHNSON, Alison Ruth","Carr Bank House, Carr Bank, Otley, United King...",https://find-and-update.company-information.se...
274,397,JORVIK REMOVALS & SELF STORAGE LTD,"HUGHES, Melissa Jade","2b Kettlestring Lane, York, England, YO30 4XF",https://find-and-update.company-information.se...
275,398,J SIMPSON SELF STORAGE LIMITED,"SIMPSON, John Christopher","Hill Top East Caldwell, Richmond, North Yorksh...",https://find-and-update.company-information.se...
276,399,J28 SELF STORAGE LIMITED,"KOHLMAN, David Peter","Ruwmell Hall, Rumwell, Taunton, Somerset, TA4 1EL",https://find-and-update.company-information.se...


In [69]:
# Exports `companies_links` DataFrame to Excel file named Companies Links
all_self_storage_companies_UK.to_excel('.\All Self Storage Companies in the UK.xlsx', index = False)

### Remarks:
* This project involved: Web Crawling, Web Scraping, Data cleaning, Python, Programming, Pandas, NumPy, Beautiful soup, Selenium, Jupyter notebook, Chrome

* Project was **_successfuly completed & delivered_** on **_21st August, 2021_**

*NB: Selenium driver must be compatible with version of Chrome browser*
